In [ ]:
pip install lime

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score
import pickle
from lime.lime_text import LimeTextExplainer
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

Preprocessing:

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords, lemmatizer, and punctuation
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
punctuation = set(string.punctuation)

In [ ]:
# Function to preprocess summarized texts
def preprocess_text(text):
    text = text.lower()  #Lowercasing
    words = word_tokenize(text) #Tokenization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and word not in punctuation] #Stopwords and punctuation removal, Lemmatization
    return ' '.join(words)

# Load and preprocess the data
df = pd.read_excel('/content/Abstractive_english_lime.xlsx') #Loading summarized judgements dataset
df['Processed_abstractive_Summaries'] = df['Abstractive Summarized Judgements'].apply(preprocess_text)
df = df[['Processed_abstractive_Summaries', 'Judgement Status']]

Generating the Embeddings (in this case InLegalBERT)

In [ ]:
# Initialize InLegalBERT tokenizer and model to generate embeddings
tokenizer = AutoTokenizer.from_pretrained('law-ai/InLegalBERT')
inlegalbert_model = AutoModel.from_pretrained('law-ai/InLegalBERT')

# Function to convert text to InLegalBERT embeddings
def inlegalbert_embed(text, model, tokenizer, max_len=512):
    tokens = tokenizer(text, return_tensors='pt', max_length=max_len, truncation=True, padding='max_length')
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Generate InLegalBERT embeddings
df['embeddings'] = df['Processed_abstractive_Summaries'].apply(lambda x: inlegalbert_embed(x, inlegalbert_model, tokenizer))


In [ ]:
# Create a DataFrame of embeddings
embeddings = np.vstack(df['embeddings'].values)
embeddings_df = pd.DataFrame(embeddings)
embeddings_df['Judgement Status'] = df['Judgement Status'].values

X = embeddings_df.drop(columns='Judgement Status')
y = embeddings_df['Judgement Status']

# Split data into train and test sets (75:25)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Base Models

In [13]:
base_models = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'DecisionTree': DecisionTreeClassifier(),
    'XGB': XGBClassifier(),
    'LGBM': LGBMClassifier(),
    'MLP': MLPClassifier(),
    'KNN': KNeighborsClassifier(),
    'GaussianNB': GaussianNB()
}

In [14]:
# Defining hyperparameter grids for each model
param_grids = {
    'RandomForest': {
        'n_estimators': [50, 100, 150],
        'max_features': ["sqrt", "log2"],
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [2, 3, 5, 7, 10],
        'min_samples_leaf': [1, 2, 3, 4, 5]
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto', 0.1, 1, 10],
        'degree': [2, 3, 4],
        'coef0': [0.0, 0.1, 0.5, 1.0]
    },
    'DecisionTree': {
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [2, 3, 5, 10],
        'min_samples_leaf': [1, 2, 4, 5]
    },
    'XGB': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'min_child_weight': [1, 3, 5]
    },
    'LGBM': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'min_child_samples': [10, 20,30]
    },
    'MLP': {
         'hidden_layer_sizes': [(50, 50), (100, 100), (50,)],
         'alpha': [0.0001, 0.001, 0.01, 0.1],
         'max_iter': [200, 300, 400, 500],
         'activation': ['logistic', 'tanh', 'relu']
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    'GaussianNB': {}
}

Hyperparameter tuning with GridSearchCV

In [ ]:
train_meta_features = []
test_meta_features = []
column_names = []

performance_metrics = []

# Perform hyperparameter tuning for each model
for model_name, model in base_models.items():
    print(f"Tuning hyperparameters for {model_name}...")

    param_grid = param_grids.get(model_name, {})

    # If param_grid is not empty, use GridSearchCV
    if param_grid:
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")
    else:
        # If no hyperparameters to tune (e.g., GaussianNB), fit directly
        best_model = model.fit(X_train, y_train)

    train_preds = best_model.predict(X_train).reshape(-1, 1)
    test_preds = best_model.predict(X_test).reshape(-1, 1)

    train_meta_features.append(train_preds)
    test_meta_features.append(test_preds)
    column_names.append(f'predicted_{model_name}')

    train_accuracy = accuracy_score(y_train, train_preds)
    test_accuracy = accuracy_score(y_test, test_preds)
    train_f1 = f1_score(y_train, train_preds, average='weighted')
    test_f1 = f1_score(y_test, test_preds, average='weighted')

    performance_metrics.append({
        'Model': model_name,
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy,
        'Train F1 Score': train_f1,
        'Test F1 Score': test_f1
    })

performance_metrics_df = pd.DataFrame(performance_metrics)

# Save the performance metrics to an Excel file
performance_metrics_df.to_excel('performance_metrics.xlsx', index=False)

train_meta_features = np.hstack(train_meta_features)
test_meta_features = np.hstack(test_meta_features)

train_meta_df = pd.DataFrame(train_meta_features, columns=column_names)
test_meta_df = pd.DataFrame(test_meta_features, columns=column_names)

train_meta_df['Judgement Status'] = y_train.values
test_meta_df['Judgement Status'] = y_test.values

combined_meta_df = pd.concat([train_meta_df, test_meta_df])
combined_meta_df.to_excel('combined_predictions.xlsx')

Meta-Models

In [15]:
meta_df = pd.read_excel('/content/combined_predictions.xlsx')

X_meta = meta_df.drop(columns='Judgement Status')
y_meta = meta_df['Judgement Status']

# Split meta_model_data into train and test sets (75:25)
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(X_meta, y_meta, test_size=0.25, random_state=29)

In [ ]:
# Defining the meta-models with the best hyperparameters (from previous GridSearchCV)
meta_models = {
    'rf': RandomForestClassifier(
        n_estimators=50,
        max_depth=10,
        min_samples_split=3,
        min_samples_leaf=1,
        max_features='sqrt'
    ),
    'svm': SVC(
        C=10,
        kernel='poly',
        degree=2,
        gamma='scale',
        coef0=1.0
    ),
    'dt': DecisionTreeClassifier(
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=3
    ),
    'xgb': XGBClassifier(
        max_depth=7,
        learning_rate=0.2,
        n_estimators=50,
        colsample_bytree=0.8,
        subsample=1,
        min_child_weight=3
    ),
    'lgbm': LGBMClassifier(
        max_depth=7,
        learning_rate=0.1,
        n_estimators=200,
        colsample_bytree=1.0,
        subsample=0.8,
        min_child_samples=30
    ),
    'mlp': MLPClassifier(
        hidden_layer_sizes=(50,),
        activation='relu',
        alpha=0.0001,
        max_iter=200
    ),
    'knn': KNeighborsClassifier(
        n_neighbors=9,
        weights='uniform',
        p=2
    ),
    'nb': GaussianNB()
}

train_meta_predictions = []
test_meta_predictions = []
column_names_meta = []
meta_model_metrics = []

for model_name, meta_model in meta_models.items():

    meta_model.fit(X_train_meta, y_train_meta)

    train_meta_preds = meta_model.predict(X_train_meta).reshape(-1, 1)
    test_meta_preds = meta_model.predict(X_test_meta).reshape(-1, 1)

    train_meta_predictions.append(train_meta_preds)
    test_meta_predictions.append(test_meta_preds)
    column_names_meta.append(f'meta_predicted_{model_name}')

    train_accuracy_meta = accuracy_score(y_train_meta, train_meta_preds)
    test_accuracy_meta = accuracy_score(y_test_meta, test_meta_preds)
    train_f1_meta = f1_score(y_train_meta, train_meta_preds, average='weighted')
    test_f1_meta = f1_score(y_test_meta, test_meta_preds, average='weighted')

    meta_model_metrics.append({
        'Model': model_name,
        'Train Accuracy': train_accuracy_meta,
        'Test Accuracy': test_accuracy_meta,
        'Train F1 Score': train_f1_meta,
        'Test F1 Score': test_f1_meta
    })

meta_model_metrics_df = pd.DataFrame(meta_model_metrics)

# Save the meta-model metrics to an Excel file
meta_model_metrics_df.to_excel('meta_model_metrics.xlsx', index=False)

train_meta_predictions = np.hstack(train_meta_predictions)
test_meta_predictions = np.hstack(test_meta_predictions)

train_meta_pred_df = pd.DataFrame(train_meta_predictions, columns=column_names)
test_meta_pred_df = pd.DataFrame(test_meta_predictions, columns=column_names)

train_meta_pred_df['Judgement Status'] = y_train.values
test_meta_pred_df['Judgement Status'] = y_test.values

combined_meta_pred_df = pd.concat([train_meta_pred_df, test_meta_pred_df])

# Saving all meta-model predictions to a single Excel file
combined_meta_pred_df.to_excel('final_predictions.xlsx', index=False)


Explainability with LIME

In [ ]:
def predict_proba_base_models(texts):
    print("Input texts to predict_proba_base_models:", texts)

    # Generate embeddings for a test input text
    embeddings = np.array([inlegalbert_embed(text, inlegalbert_model, tokenizer) for text in texts])
    print("Generated embeddings shape:", embeddings.shape)

    embeddings_df = pd.DataFrame(embeddings)
    embeddings_df.to_excel('current_embeddings_lime_generated.xlsx')

    # Generate base model predictions
    base_model_preds = [model.predict(embeddings) for _, model in base_models.values()]

    for i, preds in enumerate(base_model_preds):
        print(f"Base model {i} predictions shape:", preds.shape)

    # Stack base model predictions
    base_model_preds_stacked = np.hstack([preds.reshape(-1, 1) for preds in base_model_preds])
    print("Stacked base model predictions shape:", base_model_preds_stacked.shape)

    # Save base model predictions to a DataFrame
    base_model_preds_df = pd.DataFrame(base_model_preds_stacked, columns=[f'predicted_{name}' for name, _ in base_models.items()])
    base_model_preds_df.to_excel("current_base_model_predictions_lime_generated.xlsx")

    return meta_model.predict_proba(base_model_preds_stacked)


In [ ]:
test_df_meta = pd.concat([X_test_meta, y_test_meta], axis=1)
idx = test_df_meta.index[120] #Lime predictions for the record with index 120 in the test dataframe of meta-model
text_example = df["Processed_Extractive_Summaries"].iloc[idx]
print(text_example)

# Initialize LIME explainer
class_names = [0, 1, 2, 3]
explainer = LimeTextExplainer(class_names=class_names)

# Generate LIME explanation
exp = explainer.explain_instance(
    text_example,
    lambda x: predict_proba_base_models(x),
    num_features=15,
    top_labels=len(class_names)
)

# Print and visualize the explanation for each class
for label in class_names:
    print(f"\nExplanation for class {label}:")
    explanation_text = exp.as_list(label=label)
    print(explanation_text)

    exp.show_in_notebook(text=text_example, labels=[label])

    # Plot the LIME explanation
    weights = OrderedDict(exp.as_list(label=label))
    lime_weights = pd.DataFrame({"words": list(weights.keys()), "weights": list(weights.values())})

    # Define custom colors
    custom_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]

    # Create the bar plot with custom colors
    plt.figure(figsize=(10, 6))
    sns.barplot(x="words", y="weights", data=lime_weights, palette=custom_colors[:len(lime_weights)])
    plt.xticks(rotation=45)
    plt.title(f"Class {label} - Features Weights Given by LIME")
    plt.show()

# Save the LIME explanation object
with open('120_lime_explanation_of_all_classes.pkl', 'wb') as file:
    pickle.dump(exp, file)